# AI Overlord
## Descent: Journeys in the Dark AI Uprising

In [1]:
############### IMPORTS ###############

import numpy as np
import pandas as pd
from descent_ai import *

############### FORMATTING ###############

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', 1000)


### Notes
* Enemies in this AI version just use basic hero attacks and ignore overlord powers
* Enemies are able to battle, run, guard, etc., depending on their AI description / role

### Target / Default Target
* First check for ideal target within range = engaged
* If none, DEFAULT TARGET: Most damaged > lowest total HP > lowest defense > heros choice
* repeat with more distant ranges until a target is chosen

### Definitions
* Aim = reroll misses, surges = range
* Battle = make 2 attacks
* Berserk = if the monster has one or more wound tokens on it, it rolls all 5 power dice when attacking
* Dodge = reroll all non-misses on dice with miss icons
* Drain = each HP of drain is treated as a leech attack (-surges, heal monster)
* Fear = attacker must spend 1 surge for every Fear rank or the attack misses
* Flee = move max distance away from characters, maximize distance between monsters
* Form a line = 1st monster ends turn at range to target closest to obstruction/wall, subsequent enemies continue line to block maximum movement, starting at a wall when possible, prioritising the line over their preferred target when targets can be substituted
* Guard = makes an interrupt attack when a hero enters range
* Leech = for each wound lost due to a Leech attack, the target also loses 1 fatigue (or suffers 1 additional wound, ignoring armor, if out of fatigue) and the attacker is healed of 1 wound
* Range = the range at which enemies attacks are optimal (note: enemy orders include ±1 range when checking if "at range")
* Run = move up to 2x speed
* Stealth = like an invisibility potion, roll stealth die to attack, every turn roll power die - surge means no more stealth
* Swarm = a figure with Swarm may roll 1 extra power die for every other friendly figure adjacent to its target
* Swarm = attack the swarm target, attacks gain the swarm effect
* Undying n = Undying effect (roll power die, surge is ressurect) using n dice


In [2]:
# import moster dataframe
MONSTERS = MONSTER_DF.copy()
MONSTERS.name = MONSTERS.name.str.replace(' Master', '').str.replace(' ', '_').str.lower()
MONSTERS = pd.concat([
    MONSTERS.loc[MONSTERS['rank'] != 'master', ['name', 'expansion', 'abilities', 'attack_type', 'range', 'movement']].reset_index(drop=True), 
    MONSTERS.groupby('name').sum('quantity')[['quantity']].reset_index()
    ], axis=1
).iloc[:, [0, 3, 7, 1, 4, 5, 2]]
MONSTERS

,name,attack_type,quantity,expansion,range,movement,abilities
0,bane_spider,magic,9,base,1,5,Poison
1,beastman,melee,9,base,0,4,+1 Damage
2,blood_ape,melee,6,AoD,0,4,Leap
3,chaos_beast,unknown,3,AoD,4,3,"Morph, Sorcery 1"
4,dark_priest,magic,6,AoD,2,4,Dark Prayer
5,deep_elf,melee,3,AoD,0,5,"Shadowcloak, Pierce 3"
6,demon,magic,2,base,7,3,"Aura 1, Sorcery 2, Fear 1"
7,dragon,ranged,2,base,3,4,"Burn, Breath, Pierce 5"
8,ferrox,melee,6,WoD,0,4,Bleed
9,giant,melee,2,base,0,3,"Reach, Stun"


In [3]:
# import archetype data
ARCHETYPES = pd.read_csv('archetype_import.csv')
ARCHETYPES.T.fillna('')

,0,1,2,3,4,5,6,7,8,9,10,11,12
archetype,berserker,bombardier,controller,minion,normal,skirmisher,sniper,soldier,stalker,swarm,tank,thief,undead
type_melee,True,,,True,True,True,,True,True,True,True,True,True
type_ranged,,True,,True,True,True,True,,True,,,True,True
type_magic,,True,True,True,True,True,True,,True,,,,True
type_boss,True,True,True,,True,True,True,True,True,,True,,True
target,closest,"max targets, min current HP","max targets, highest movement",closest,closest,closest engaged,max base HP,closest unengaged,min current HP,"engaged, closest",max base HP,most gold,closest
range_modifier,,"(0, 2)","(0, 2)",,,,"(2, 4)",,,,,,
move_modifier,,,,,,"(3, 3)","(0, 3)",,"(0, 3)",,,,
special,"gains ""berserk"" at half HP",blast +1,stun,"2x spawn, 1HP",,"+3 move, swarm command target","+2 range, swarm command target",form a line starting form first target,"starts with stealth, +5 power dice for attacks in stealth",swarm,+2 armor,"starts with stealth, stealth again at half HP",surges drain HP
boss_special,"always battle, berserk","2+ range, blast +2","2+ range, stun, fear 2",,,"+3 damage, +3 move, command","+4 range, +3 move, command","+3 armor, +3 pierce","+5 pierce, +3 move, stealth",,+4 armor,,"undying 2, leech"


In [4]:
# example monsters / encounters
m = Monsters(MONSTER_DF)
print('Monsters')
print(m.quest_monsters)
print(m.quest_boss, 'Boss')
print()
print('Encounters')
for encounter in m.encounters.values():
    print(encounter)

Monsters
['Ice Wyrm', 'Ogre', 'Sorcerer']
Ogre Boss

Encounters
['Ice Wyrm', 'Ogre', 'Ogre']
['Ice Wyrm', 'Ice Wyrm', 'Sorcerer Master']
['Ice Wyrm Master', 'Ogre Master', 'Sorcerer', 'Sorcerer', 'Sorcerer Master']
['Ogre Master', 'Ogre Boss']


In [59]:
class MonsterAI:
    # TODO: for now master and minions have the same archetype, it is possible to tweak this

    def __init__(self, monster_name, archetype=None, boss=False):
        if monster_name not in list(MONSTERS.name):
            raise ThisIsNotAMonsterYouCanChoose
        self.monster_name = monster_name
        self.boss = boss
        self._archetype_data = self.get_archetype_data(archetype)

    def get_archetype_data(self, archetype):
        columns = ['archetype', 'target', 'range_modifier', 'move_modifier', 'special', 'boss_special', 'lt_range', 'at_range', 'gt_range', 'gt_move_range']
        all_eligible_archetypes = ARCHETYPES.loc[ARCHETYPES[self.monster_name].notna(), columns]
        if archetype:
            if archetype not in list(all_eligible_archetypes['archtype']):
                raise ThisIsNotAReasonableChoiceForThatMonsterError
            return all_eligible_archetypes[all_eligible_archetypes['archtype']==archetype]
        return all_eligible_archetypes.sample()

    @property
    def archetype(self):
        return self._archetype_data['archetype'].iloc[0]

    @property
    def target(self):
        return self._archetype_data['target'].iloc[0]
    
    @property
    def range(self):
        range = MONSTERS.loc[MONSTERS['name']==self.monster_name, 'range'].iloc[0]
        if self._archetype_data['range_modifier'].notnull().iloc[0]:
            if self.boss:
                range += eval(self._archetype_data['range_modifier'].iloc[0])[1]
            else:
                range += eval(self._archetype_data['range_modifier'].iloc[0])[0]
        return range

    @property
    def movement(self):
        movement = MONSTERS.loc[MONSTERS['name']==self.monster_name, 'movement'].iloc[0]
        if self._archetype_data['move_modifier'].notnull().iloc[0]:
            if self.boss:
                movement += eval(self._archetype_data['move_modifier'].iloc[0])[1]
            else:
                movement += eval(self._archetype_data['move_modifier'].iloc[0])[0]
        return movement
    
    @property
    def lt_range(self):
        # TODO: check magic / ranged and plus minus 1
        return self._archetype_data['lt_range'].iloc[0]
    
    @property
    def at_range(self):
        # TODO: check magic / ranged and plus minus 1
        return self._archetype_data['at_range'].iloc[0]
    
    @property
    def gt_range(self):
        # TODO: check magic / ranged and plus minus 1
        return self._archetype_data['gt_range'].iloc[0]
    
    @property
    def gt_move_range(self):
        # TODO: check magic / ranged and plus minus 1
        return self._archetype_data['gt_move_range'].iloc[0]

    @property
    def special(self):
        if self.boss:
            return self._archetype_data['boss_special'].iloc[0]
        return self._archetype_data['special'].iloc[0]

    def boss_stuff(self):
        pass

    def summary(self):
        print('================================================================')
        print(f"{self.archetype} {self.monster_name.replace('_', ' ')}".replace('normal ', '').title())
        print('================================================================\n')

        if pd.notnull(self.special):
            print('Special Powers:', self.special.title().replace('Hp', 'HP'))

        print('Target:', self.target.title())

        attack_type = MONSTERS.loc[MONSTERS['name']==self.monster_name, 'attack_type'].iloc[0]
        if attack_type == 'melee':
            attack_range = 1 if 'reach' in MONSTERS.loc[MONSTERS.name==self.monster_name, 'abilities'].iloc[0].lower() else 0
            print('Attack Range:', attack_range, '\n')
        else:
            attack_range = self.range
            print('Attack Range:', attack_range-1, 'to', attack_range+1, '\n')

        print('Combat AI Instructions')
        print('----------------------')

        if attack_type == 'melee':
            if self.at_range:
                print('If at attack range: ', self.at_range)
            if self.gt_range:
                print(f'If within {self.movement + attack_range} spaces: ', self.gt_range)
            if self.gt_move_range:
                print(f'Otherwise: ', self.gt_move_range)
        else:
            if self.lt_range:
                print(f'If closer than {self.range-1} spaces: ', self.lt_range)
            if self.at_range:
                print(f'If {attack_range-1} to {attack_range+1} spaces: ', self.at_range)
            if self.gt_range:
                print(f'If within {self.movement + attack_range} spaces: ', self.gt_range)
            if self.gt_move_range:
                print('Otherwise: ', self.gt_move_range)
        
        print('================================================================\n')



In [61]:
monster_name = MONSTERS.sample().name.iloc[0]
m = MonsterAI(monster_name)

m.summary()

Sniper Sorcerer

Special Powers: +2 Range, Swarm Command Target
Target: Max Base Hp
Attack Range: 6 to 8 

Combat AI Instructions
----------------------
If closer than 6 spaces:  flee
If 6 to 8 spaces:  aim > attack
If within 11 spaces:  move to range > attack
Otherwise:  stay move away from closest hero



# OLD METHOD, NOT GREAT

In [8]:
############### MONSTER STRATEGY ###############

def monster_order(chance_heroes_choose=0.1):

    monster_order = {
        'Monster Order by Type:': ['melee', 'ranged'],
        'Monster Order by Rank:': ['masters', 'minion']
    }
    
    dataframe = pd.DataFrame()
    for key in monster_order.keys():
        order = ''
        for item in np.random.permutation(monster_order[key]):
            if order:
                order += ' => ' + item.upper()
            else:
                order += item.upper()
        
        if np.random.random() < chance_heroes_choose: 
            order = 'HEROES CHOOSE'
        
        dataframe.loc[key, ''] = order

    return dataframe


# # formatted test output
# print('\n==================================\nMONSTER ORDER OF ATTACK')
# display(monster_order(0.5).style.set_properties(**{'text-align': 'left'}))
# print('\n')

In [9]:
############### OVERLORD CARDS ###############

def overlord_card_rule(draw_card_manually=True, max_cards=5):
    
    # overlord card rules
    overlord_rules = {
        'What to do with the overlord card': [
            'ignore and discard', 
            'play first time possible, discard', 
            'play first time possible, discard, draw a new card, repeat',
            'when possible, roll red, play the card if you roll a Surge, then keep card and repeat for the round',
            'when possible, roll red, play the card if you roll a Surge, then discard the card'
        ]
    }
    
    card_rule = np.random.choice(overlord_rules['What to do with the overlord card'])
    print(card_rule)
    
    if draw_card_manually or 'ignore and discard' in card_rule:
        pass
    else:
        if 'draw a new card' not in card_rule:  # only draw more than one card if max cards
            min_cards, max_cards = 1, 1
        else:
            print('\n...once you run out of cards, you are done')
            min_cards = 2
        
        display(overlord_card(min_cards=min_cards, max_cards=max_cards))


def overlord_card(min_cards=1, max_cards=1):
    
    if min_cards > max_cards:
        max_cards = min_cards
    
    # choose random card
    overlord_cards = {
        'Pit Trap': 'Play this card when a hero enters an empty space. He tests Awareness. If he fails, he suffers 1 Heart and loses 1 movement point. If he has no movement points to lose (such as if he suffered fatigue to move), he is Stunned.',
        'Critical Blow': 'Play this card when a monster attacks a hero, after rolling dice. \nThe attack gains: \nSurge: +3 Heart',
        'Poison Dart': 'Play this card when a hero opens a door or searches. He tests Awareness or Might (your choice). If he passes, draw 1 Overlord Card. If he fails, he suffers 1 Heart, 1 Fatigue, and he is Poisoned.',
        'Word of Misery': 'Play this card at the start of your turn. During this turn, each time a hero suffers any Heart, he also suffers 1 Fatigue in addition to the Heart suffered.',
        'Dark Charm': 'Play this card on a hero at the start of your turn. The hero tests Willpower. If he passes, draw 1 Overlord Card. If he fails, you may perform a move or attack action with that hero as if he were one of your monsters this turn. You cannot force him to suffer Fatigue or use a Potion, but you may force him to attack himself.',
        'Dark Might': 'Play this card after you roll dice for an attack. Add 1 Surge to the results.',
        'Tripwire': 'Play this card when a hero enters an empty space during a move action. He tests Awareness. If he fails, he must end his move action (he can still suffer Fatigue to move further, or perform a second move action if this was his first action).',
        'Dash': 'Play this card when activating a monster during your turn. That monster may perform an additional move action this turn in addition to its normal 2 actions.',
        'Frenzy': 'Play this card when activating a monster during your turn. That monster may perform an additional attack action this turn in addition to its normal 2 actions.',
        'Dark Fortune': 'Play this card after you roll dice. You may reroll 1 die.'
    }
    return pd.DataFrame(overlord_cards, index=['']).T.sample(np.random.randint(1, max_cards+1)).style.set_properties(**{'text-align': 'left'})


# # formatted test output
# print('\n==================================\nOVERLORD CARDS\n\ndraw a card...')
# overlord_card_rule(False)  # false means the computer draws the cards
# print('\n')

In [10]:
############### MONSTER TACTICS ###############

# choose 1 random output from lists
melee = {
    'Quest:': ['top priority', 'attack but attempt', 'ignore'],
    'Target:': ['closest', 'most damaged in range', 'most surroundable'], 
    'Move:': ['attack then back off', 'attack and stay adjacent'],
    'Special:': [
        'use special instead of attack (both if possible)', 
        'attack then special if possible', 
        'no special ability this turn'
    ],
}

# choose 1 random output from lists
ranged = {
    'Quest:': ['top priority', 'attack but attempt', 'ignore'],
    'Target:': ['closest', 'most damaged in range', 'most surroundable'],
    'Range:': ['within range 3', 'within roll-2', 'within roll-1'],
    'Move:': ['attack then back off', 'attack and stay at range'],
    'Special:': [
        'use special instead of attack (both if possible)', 
        'attack then special if possible', 
        'no special ability this turn'
    ],
}

def attack(attacker_dict):

    dataframe = pd.DataFrame()

    for key in attacker_dict.keys():
        dataframe.loc[key, ''] = np.random.choice(attacker_dict[key])

    return dataframe


# # formatted test output
# print('\n\n==================================\nMELEE ATTACKERS')
# display(attack(melee))
# print('\n\n==================================\nRANGE ATTACKERS')
# display(attack(ranged))
# print('\n')

In [11]:
############### SUMMARY OF TURN ###############

def turn_rules_summary(overlord_cards=False, draw_card_manually=True, max_cards=5, chance_heroes_choose=0.1):
    print('\n')
    print('------------------------------ MONSTER STRATEGY ------------------------------')
    order = monster_order(chance_heroes_choose)
    display(order.style.set_properties(**{'text-align': 'left'}))
    print('\n')
    if overlord_cards:
        print('------------------------------ OVERLORD CARDS ------------------------------')
        print('\ndraw a card...')
        overlord_card_rule(draw_card_manually, max_cards)  # false means the computer draws the cards
        print('\n')
    print('------------------------------ MONSTER TACTICS ------------------------------')
    if order.iloc[0].str.split()[0][0].lower() == 'melee':  # same order as order above
        print('\n------------------------------\nMELEE ATTACKERS')
        display(attack(melee))
        print('\n------------------------------\nRANGED ATTACKERS')
        display(attack(ranged))
    else:  # this includes 'heroes choose'
        print('\n------------------------------\nRANGED ATTACKERS')
        display(attack(ranged))
        print('\n------------------------------\nMELEE ATTACKERS')
        display(attack(melee))
    print('-------------------------------------------------------------------------------')

# # test
# turn_rules_summary()

In [12]:
############### GAME LOOP ###############

def game_loop(round_counter=True, overlord_cards=False, draw_card_manually=True, max_cards=5, chance_heroes_choose=0.1):
    round_count=0
    while True:
        round_count += 1
        if round_counter:
            print('##############################################################################')
            print(f'                                      ROUND {round_count}')
            print('##############################################################################')
        turn_rules_summary(overlord_cards, draw_card_manually, max_cards, chance_heroes_choose)
        # end game condition
        print('\n\nPRESS ENTER TO CONTINUE...\n\n\n\n\n')
#         time.sleep(0.3)  # just to avoid the input printing above the end of the stuff before it in jupyter
        if(input() != ''):  # anything other than enter exits (but only after an enter)
            break


# # test
# game_loop(True, True, True, 5, .5)

In [13]:
############### SUMMARY OF TURN ###############
# ALT EVEN LESS SPACE

def turn_rules_summary(overlord_cards=False, draw_card_manually=True, max_cards=5, chance_heroes_choose=0.1):
    print('\n-------------------------------------------------------------------------------')
    if overlord_cards:
        print('\ndraw a card...')
        overlord_card_rule(draw_card_manually, max_cards)  # false means the computer draws the cards
        
    order = monster_order(chance_heroes_choose)
    print(order.to_string())

    if order.iloc[0].str.split()[0][0].lower() == 'melee':  # same order as order above
        print('\nMELEE ATTACKERS')
        print(attack(melee).to_string())
        print('\nRANGED ATTACKERS')
        print(attack(ranged).to_string())
    else:  # this includes 'heroes choose'
        print('\nRANGED ATTACKERS')
        print(attack(ranged).to_string())
        print('\nMELEE ATTACKERS')
        print(attack(melee).to_string())
    

# # test
# turn_rules_summary(True)

In [14]:
# GAME
turn_rules_summary(True, True, 5, .5)


-------------------------------------------------------------------------------

draw a card...
ignore and discard
                                     
Monster Order by Type:  HEROES CHOOSE
Monster Order by Rank:  HEROES CHOOSE

RANGED ATTACKERS
                                                          
Quest:                                              ignore
Target:                                            closest
Range:                                       within roll-2
Move:                             attack and stay at range
Special:  use special instead of attack (both if possible)

MELEE ATTACKERS
                                                          
Quest:                                        top priority
Target:                                            closest
Move:                             attack and stay adjacent
Special:  use special instead of attack (both if possible)
